# Analysis B-Point Algorithms

## Setup and Helper Functions

### Imports

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.utils.dataframe_handling import multi_xs
from fau_colors import cmaps, register_fausans_font
from IPython.display import Markdown

from pepbench.data_handling import (
    add_unique_id_to_results_dataframe,
    compute_improvement_outlier_correction,
    compute_pep_performance_metrics,
    get_error_by_group,
)
from pepbench.datasets import EmpkinsDataset, GuardianDataset
from pepbench.export import (
    convert_to_latex,
    create_algorithm_result_table,
    create_nan_reason_table,
    create_outlier_correction_table,
)
from pepbench.io import load_challenge_results_from_folder
from pepbench.plotting.results import (
    boxplot_algorithm_performance,
    paired_plot_error_outlier_correction,
    regplot_error_heart_rate,
    residual_plot_pep,
    residual_plot_pep_heart_rate,
    residual_plot_pep_participant,
    residual_plot_pep_phase,
    violinplot_algorithm_performance,
)
from pepbench.utils import get_nan_reason_mapping, rename_algorithms, rename_metrics, styling

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path("../../")

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

### Input Paths

In [ ]:
result_path = root_path.joinpath("results")

### Output Paths

In [ ]:
paper_path = json.load(root_path.joinpath("paper_path.json").open(encoding="utf-8"))["paper_path"]
paper_path = Path(paper_path)

export_path = root_path.joinpath("exports")
img_path = export_path.joinpath("plots")
stats_path = export_path.joinpath("stats")

img_path_paper = paper_path.joinpath("img")
tab_path_paper = paper_path.joinpath("tab")
suppl_img_path_paper = paper_path.joinpath("supplementary_material/img")
suppl_tab_path_paper = paper_path.joinpath("supplementary_material/tab")

bp.utils.file_handling.mkdirs(
    [
        result_path,
        export_path,
        img_path,
        stats_path,
        img_path_paper,
        tab_path_paper,
        suppl_img_path_paper,
        suppl_tab_path_paper,
    ]
)

In [ ]:
algo_levels = ["b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["B-Point Algorithm", "Outlier Correction"], strict=False))

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

## EmpkinS Dataset

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_b_point"),
    index_cols_per_sample=["participant", "condition", "phase"],
)

In [ ]:
results_per_sample_empkins = results_empkins.per_sample.droplevel([0])
results_agg_total_empkins = results_empkins.agg_total.droplevel([0])
results_per_sample_empkins.head()

In [ ]:
selected_algos_for_plotting_empkins = [
    ("debski1993-second-derivative", "none"),
    ("lozano2007-linear-regression", "none"),
    ("forouzanfar2018", "none"),
    ("drost2022", "none"),
]

In [ ]:
selected_algos_for_residual_empkins = ["drost2022", "lozano2007-linear-regression", "forouzanfar2018"]

In [ ]:
selected_algos_for_residual_outlier_correction_empkins = [
    "drost2022",
    "debski1993-second-derivative",
    "forouzanfar2018",
]
outlier_algos = ["none", "linear-interpolation", "forouzanfar2018"]
outlier_algos_rename = ["None", "LinInt", "For18"]

### Results Table

In [ ]:
metrics_empkins = compute_pep_performance_metrics(results_per_sample_empkins, num_heartbeats=results_agg_total_empkins)
metrics_empkins.style.highlight_min(
    subset=["Mean Absolute Error [ms]", "Mean Absolute Relative Error [%]"], props="background-color: PaleGreen;"
)

In [ ]:
metrics_empkins_table = metrics_empkins.xs("none", level="outlier_correction_algorithm")
metrics_empkins_table.style.highlight_min(
    subset=["Mean Absolute Error [ms]", "Mean Absolute Relative Error [%]"], props="background-color: PaleGreen;"
)

In [ ]:
result_table = create_algorithm_result_table(metrics_empkins_table)

latex_output = convert_to_latex(
    result_table,
    collapse_index_columns=False,
    column_header_bold=True,
    column_format="p{1.0cm}S[table-format=1.1(2)]S[table-format=1.1(2)]S[table-format=1.1(2)]p{1.75cm}",
    caption=r"Results of the B-point extraction algorithms (without outlier correction) on the \textit{EmpkinS Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order.",
    label="tab:b_point_results_empkins",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"{Invalid", r"{\bfseries Invalid")
# some manual post-processing
latex_output = latex_output.replace(
    r"{} & {\bfseries \ac{MAE} [ms]}", r"{\bfseries B-point Algorithm} & {\bfseries \ac{MAE} [ms]}"
)
latex_output = latex_output.replace(r"{B-Point Detection} & {} & {} & {} & {} \\", "")


tab_path_paper.joinpath("tab_b_point_results_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

### Plots

In [ ]:
results_empkins_plot = multi_xs(
    data=results_per_sample_empkins, keys=selected_algos_for_plotting_empkins, level=algo_levels
)
results_empkins_plot = results_empkins_plot.droplevel("outlier_correction_algorithm")
results_empkins_plot = results_empkins_plot.reindex([s[0] for s in selected_algos_for_plotting_empkins], level=0)
results_empkins_plot.head()

#### Absolute Error

In [ ]:
fig, ax = boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

fig.savefig(img_path.joinpath("img_boxplot_b_point_algorithms_mae_empkins.pdf"), transparent=True)

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    figsize=(6, 5),
)

#### Absolute Error (with and without Outlier)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 3))

boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=True,
    width=0.9,
    title="B-Point Detection Results – With Outlier",
    fig=fig,
    ax=axs[0],
)
boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="B-Point Detection Results – Without Outlier",
    fig=fig,
    ax=axs[1],
)
for ax in axs:
    ax.set_xlabel(None)
fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_boxplot_b_point_algorithms_mae_with_without_outlier_empkins.pdf"), transparent=True)

#### Error

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_empkins_plot,
    metric="error_per_sample_ms",
    figsize=(6, 5),
)

#### Error per Participant

In [ ]:
error_per_participant_empkins = get_error_by_group(results_per_sample_empkins, grouper="participant")
error_per_participant_empkins = multi_xs(
    error_per_participant_empkins, selected_algos_for_plotting_empkins, level=algo_levels, axis=1
)
error_per_participant_empkins = error_per_participant_empkins.round(2)
error_per_participant_empkins = error_per_participant_empkins.rename(columns=rename_algorithms).rename(
    columns=rename_metrics
)

error_per_participant_empkins.style.highlight_max(props="background-color: Pink;")

In [ ]:
latex_output = convert_to_latex(
    error_per_participant_empkins.style.highlight_max(props="background-color: Pink;").format_index(
        escape="latex", axis=0
    ),
    collapse_index_columns=False,
    column_header_bold=True,
    column_format="p{3.5cm}" + "S[table-format=2.2]" * len(error_per_participant_empkins.columns),
    caption=r"Mean Abolute Error of selected B-point extraction algorithms on the \textit{EmpkinS Dataset} per participant. The values with the highest errors are highlighted in red.",
    label="tab:b_point_results_per_participant_empkins",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"\begin{table}[ht]", r"\begin{table}[ht]\small")
latex_output = latex_output.replace(r"b_point_algorithm", r"\bfseries B-point Algorithm")
latex_output = latex_output.replace(r"outlier_correction_algorithm", r"\bfseries Outlier Correction Algorithm")
latex_output = latex_output.replace(r"{participant}", r"{Participant}")
latex_output = latex_output.replace(r"{metric}", r"{}")
latex_output = latex_output.replace(r"{\bfseries mean}", r"{Mean}")
latex_output = latex_output.replace(r"{\bfseries std}", r"{SD}")
latex_output = latex_output.replace(r"{std}", r"{SD}")
latex_output = latex_output.replace(r"\sisetup{", r"\sisetup{round-mode=places,round-precision=2,")

suppl_tab_path_paper.joinpath("tab_b_point_results_per_participant_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Residual Plots

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep(
        results_empkins_plot,
        selected_algos_for_residual_empkins[i],
        alpha=0.5,
        show_upper_limit=True,
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)
axs[0].set_ylim([-125, 150])

fig.tight_layout()

for path in [img_path, img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep_participant(
        results_empkins_plot,
        selected_algos_for_residual_empkins[i],
        alpha=0.5,
        show_upper_limit=True,
        annotate_fontsize="small",
        annotate_bbox=True,
        show_legend=False,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)
axs[0].set_ylim([-125, 150])

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_per_participant_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep_phase(
        results_empkins_plot,
        selected_algos_for_residual_empkins[i],
        alpha=0.5,
        show_upper_limit=True,
        show_legend=(i == 0),
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)

axs[0].set_ylim([-125, 150])

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_per_phase_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(11, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep_heart_rate(
        results_empkins_plot,
        selected_algos_for_residual_empkins[i],
        alpha=0.5,
        show_upper_limit=True,
        show_legend=(i == 0),
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)

axs[0].set_ylim([-125, 150])


for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_heart_rate_empkins.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(11, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = regplot_error_heart_rate(
        results_empkins_plot,
        selected_algos_for_residual_empkins[i],
        error_metric="absolute_error_per_sample_ms",
        add_corr_coeff=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_b_point_error_heart_rate_empkins.pdf"), transparent=True)

#### Effect of Outlier Correction on Estimation Error

In [ ]:
metrics_empkins_outlier = create_outlier_correction_table(metrics_empkins)
metrics_empkins_outlier_style = metrics_empkins_outlier.style.apply(
    styling.highlight_outlier_improvement, subset=["Mean Absolute Error [ms]", "Invalid PEPs"]
)
metrics_empkins_outlier_style

##### LaTeX

In [ ]:
latex_output = convert_to_latex(
    metrics_empkins_outlier_style,
    collapse_index_columns=False,
    column_header_bold=True,
    siunitx=True,
    convert_css=True,
    column_format="p{1.5cm}p{1.5cm}"
    + ("S[table-column-width=0.75cm]" * (len(metrics_empkins_outlier_style.columns) - 3))
    + "p{1.0cm}" * 3,
    caption=r"Effect of Outlier Correction algorithms on the B-point extraction algorithms for the \textit{EmpkinS Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order. Resuls highlighted in \textcolor{LightGreen}{green} indicate an improvement of the metric through outlier correction, \textcolor{Pink}{red} indicate no improvement.",
    label="tab:outlier_correction_results_full_empkins",
)

# some manual post processing of latex output
latex_output = latex_output.replace(r"\sisetup{", r"\sisetup{round-mode=places,round-precision=2,")
latex_output = latex_output.replace(r"\bfseries \bfseries", r"\bfseries")
latex_output = latex_output.replace(r"\bfseries \bfseries", r"\bfseries")
latex_output = latex_output.replace(r"\bfseries \bfseries", r"\bfseries")
latex_output = latex_output.replace(r"\multicolumn{2}{r}", r"\multicolumn{2}{c}")
latex_output = latex_output.replace(r"Mean Absolute Error [ms]", r"MAE [ms]")
latex_output = latex_output.replace(r"Mean Error [ms]", r"ME [ms]")
latex_output = latex_output.replace(r"Mean Absolute Relative Error [\%]", r"MARE [\%]")
latex_output = latex_output.replace(r"{B-Point Detection}", r"{B-Point\newline Detection}")

suppl_tab_path_paper.joinpath("tab_outlier_correction_results_full_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Horizontal Table

In [ ]:
metrics_empkins_outlier_unstack = metrics_empkins_outlier[["Mean Absolute Error [ms]"]]
metrics_empkins_outlier_unstack = (
    metrics_empkins_outlier_unstack.unstack(sort=False).reorder_levels([0, 2, 1], axis=1).sort_index(axis=1)
)
metrics_empkins_outlier_unstack = metrics_empkins_outlier_unstack.reindex(outlier_algos_rename, level=1, axis=1)
metrics_empkins_outlier_unstack

##### To LaTeX

In [ ]:
result_table = create_algorithm_result_table(metrics_empkins[["Mean Absolute Error [ms]"]])
result_table = result_table.unstack("Outlier Correction").reindex(result_table.xs("None", level=-1).index, level=0)
result_table = result_table.reindex(outlier_algos_rename, level="Outlier Correction", axis=1)

latex_output = convert_to_latex(
    result_table.style.apply(styling.highlight_min_uncertainty, axis=1),
    collapse_index_columns=False,
    column_header_bold=True,
    siunitx=False,
    column_format="p{1.0cm}p{1.5cm}p{1.5cm}p{1.5cm}",
    caption=r"\ac{MAE} of the Outlier Correction algorithms on the B-point extraction algorithms on the \textit{EmpkinS Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order. The lowest \ac{MAE} values per algorithm are highlighted in \textbf{bold}. \ac{MAE} values are provided in milliseconds as (\(M\,\pm\,SD\)).",
    label="tab:outlier_correction_results_empkins",
)

# some manual post-processing
latex_output = latex_output.replace(
    r"\multicolumn{3}{r}{\bfseries Mean Absolute Error [ms]}",
    r"\multicolumn{3}{l}{\bfseries Outlier Correction Algorithm}",
)
latex_output = latex_output.replace(r"Outlier Correction & ", r"{\bfseries B-point Algorithm} & ")
latex_output = latex_output.replace(r"B-Point Detection &  &  &  \\", r"")
latex_output = latex_output.replace(r" \pm ", r"\(\pm\)")

tab_path_paper.joinpath("tab_outlier_correction_results_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Outlier Correction Residual Plots

In [ ]:
for algo in selected_algos_for_residual_outlier_correction_empkins:
    fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

    results_per_algorithm_plot_empkins = results_per_sample_empkins.xs(
        algo, level="b_point_algorithm", drop_level=False
    )

    for i, outlier_algo in enumerate(outlier_algos):
        selected_algo = [algo, outlier_algo]
        residual_plot_pep(
            results_per_sample_empkins, selected_algo, ax=axs[i], show_upper_limit=True, annotate_fontsize="small"
        )
        if i != 0:
            axs[i].set_ylabel(None)
        axs[i].set_ylim([-125, 150])

    fig.tight_layout()
    for path in [img_path, suppl_img_path_paper]:
        fig.savefig(
            path.joinpath(f"img_residual_plots_b_point_outlier_correction_{algo}_empkins.pdf"), transparent=True
        )

In [ ]:
dv = "absolute_error_per_sample_ms"

for algo in selected_algos_for_residual_outlier_correction_empkins:
    results_per_algorithm_plot_empkins = results_per_sample_empkins.xs(algo, level="b_point_algorithm")
    data_plot_paired = add_unique_id_to_results_dataframe(results_per_algorithm_plot_empkins[[dv]])

    outlier_algo_combis = [(outlier_algos[0], outlier_algos[1]), (outlier_algos[0], outlier_algos[2])]

    fig, axs = plt.subplots(ncols=2, figsize=(4, 4), sharey=True)
    fig, axs = paired_plot_error_outlier_correction(
        data=data_plot_paired, outlier_algo_combis=outlier_algo_combis, dv=dv, title=algo, axs=axs
    )

    display(Markdown(f"**B-point Algorithm**: {algo}"))
    for outlier_algo in outlier_algo_combis:
        display(Markdown(f"""**Outlier Correction Algorithms**: {' vs. '.join(outlier_algo)}"""))
        display(compute_improvement_outlier_correction(data_plot_paired, outlier_algo))

    fig.tight_layout()
    for path in [img_path, suppl_img_path_paper]:
        fig.savefig(
            path.joinpath(f"img_paired_plot_b_point_outlier_correction_improvement_{algo}_empkins.pdf"),
            transparent=True,
        )

### `NaN` Reason Table

In [ ]:
nan_reason_table_empkins = create_nan_reason_table(
    results_per_sample_empkins, outlier_algos=outlier_algos, use_short_names=True
)
nan_reason_table_empkins.head()

##### To LaTeX

In [ ]:
latex_output = convert_to_latex(
    nan_reason_table_empkins,
    column_format="p{1.5cm}" * 2 + "p{1.0cm}" * len(nan_reason_table_empkins.columns),
    column_header_bold=True,
    escape_columns=True,
    caption=r"Overview of invalid PEP reasons for different B-point algorithms on the \textit{EmpkinS Dataset}. Abbreviations: "
    + ", ".join([rf"\textit{{{k}}}: {v}" for k, v in get_nan_reason_mapping().items()]),
    label="tab:nan_reasons_empkins",
)

# some manual post-processing
latex_output = latex_output.replace(r"\centering", r"\small\centering")
latex_output = latex_output.replace(r"{Reason}", r"{\bfseries Reason}")

suppl_tab_path_paper.joinpath("tab_b_point_nan_reason_empkins.tex").open(mode="w+").write(latex_output)
print(latex_output)

## Guardian Dataset

In [ ]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

In [ ]:
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_b_point"),
    index_cols_per_sample=["participant", "phase"],
)

In [ ]:
results_per_sample_guardian = results_guardian.per_sample.droplevel([0])
results_agg_total_guardian = results_guardian.agg_total.droplevel([0])
results_per_sample_guardian.head()

In [ ]:
selected_algos_for_plotting_guardian = [
    ("debski1993-second-derivative", "none"),
    ("lozano2007-linear-regression", "none"),
    ("forouzanfar2018", "none"),
    ("drost2022", "none"),
]

In [ ]:
selected_algos_for_residual_guardian = ["drost2022", "debski1993-second-derivative", "forouzanfar2018"]

In [ ]:
selected_algos_for_residual_outlier_correction_guardian = [
    "drost2022",
    "debski1993-second-derivative",
    "forouzanfar2018",
]
outlier_algos = ["none", "linear-interpolation", "forouzanfar2018"]

### Results Table

In [ ]:
metrics_guardian = compute_pep_performance_metrics(
    results_per_sample_guardian, num_heartbeats=results_agg_total_guardian
)
metrics_guardian.style.highlight_min(
    subset=["Mean Absolute Error [ms]", "Mean Absolute Relative Error [%]"], props="background-color: LightGreen;"
)

In [ ]:
metrics_guardian_table = metrics_guardian.xs("none", level="outlier_correction_algorithm")
metrics_guardian_table.style.highlight_min(
    subset=["Mean Absolute Error [ms]", "Mean Absolute Relative Error [%]"], props="background-color: LightGreen;"
)

In [ ]:
result_table = create_algorithm_result_table(metrics_guardian_table)

latex_output = convert_to_latex(
    result_table,
    collapse_index_columns=False,
    column_header_bold=True,
    column_format="p{1.0cm}S[table-format=1.1(2)]S[table-format=1.1(2)]S[table-format=1.1(2)]p{1.75cm}",
    caption=r"Results of the B-point extraction algorithms (without outlier correction) on the \textit{Guardian Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order.",
    label="tab:b_point_results_guardian",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"{Invalid", r"{\bfseries Invalid")
# some manual post-processing
latex_output = latex_output.replace(
    r"{} & {\bfseries \ac{MAE} [ms]}", r"{\bfseries B-point Algorithm} & {\bfseries \ac{MAE} [ms]}"
)
latex_output = latex_output.replace(r"{B-Point Detection} & {} & {} & {} & {} \\", "")


tab_path_paper.joinpath("tab_b_point_results_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

### Plots

In [ ]:
results_guardian_plot = multi_xs(
    data=results_per_sample_guardian, keys=selected_algos_for_plotting_guardian, level=algo_levels
)
results_guardian_plot = results_guardian_plot.droplevel("outlier_correction_algorithm")
results_guardian_plot = results_guardian_plot.reindex([s[0] for s in selected_algos_for_plotting_guardian], level=0)
results_guardian_plot.head()

#### Absolute Error

In [ ]:
fig, ax = boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

fig.savefig(img_path.joinpath("img_boxplot_b_point_algorithms_mae_guardian.pdf"), transparent=True)

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    figsize=(6, 5),
)

#### Absolute Error (with and without Outlier)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 3))

boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=True,
    width=0.9,
    title="B-Point Detection Results – With Outlier",
    fig=fig,
    ax=axs[0],
)
boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="B-Point Detection Results – Without Outlier",
    fig=fig,
    ax=axs[1],
)
for ax in axs:
    ax.set_xlabel(None)
fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_boxplot_b_point_algorithms_mae_with_without_outlier_guardian.pdf"), transparent=True)

#### Error

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_guardian_plot,
    metric="error_per_sample_ms",
    figsize=(6, 5),
)

#### Error per Participant

In [ ]:
error_per_participant_guardian = get_error_by_group(results_per_sample_guardian, grouper="participant")
error_per_participant_guardian = multi_xs(
    error_per_participant_guardian, selected_algos_for_plotting_guardian, level=algo_levels, axis=1
)
error_per_participant_guardian = error_per_participant_guardian.round(2)
error_per_participant_guardian = error_per_participant_guardian.rename(columns=rename_algorithms).rename(
    columns=rename_metrics
)

error_per_participant_guardian.style.highlight_max(props="background-color: Pink;")

In [ ]:
latex_output = convert_to_latex(
    error_per_participant_guardian.style.highlight_max(props="background-color: Pink;").format_index(
        escape="latex", axis=0
    ),
    collapse_index_columns=False,
    column_header_bold=True,
    column_format="p{3.5cm}" + "S[table-format=2.2]" * len(error_per_participant_guardian.columns),
    caption=r"Mean Abolute Error of selected B-point extraction algorithms on the \textit{Guardian Dataset} per participant. The values with the highest errors are highlighted in red.",
    label="tab:b_point_results_per_participant_guardian",
)

# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"\begin{table}[ht]", r"\begin{table}[ht]\footnotesize")
latex_output = latex_output.replace(r"b_point_algorithm", r"\bfseries B-point Algorithm")
latex_output = latex_output.replace(r"outlier_correction_algorithm", r"\bfseries Outlier Correction Algorithm")
latex_output = latex_output.replace(r"{participant}", r"{Participant}")
latex_output = latex_output.replace(r"{metric}", r"{}")
latex_output = latex_output.replace(r"{\bfseries mean}", r"{Mean}")
latex_output = latex_output.replace(r"{\bfseries std}", r"{SD}")
latex_output = latex_output.replace(r"{std}", r"{SD}")
latex_output = latex_output.replace(r"\sisetup{", r"\sisetup{round-mode=places,round-precision=2,")

suppl_tab_path_paper.joinpath("tab_b_point_results_per_participant_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Residual Plots

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep(
        results_guardian_plot,
        selected_algos_for_residual_guardian[i],
        alpha=0.5,
        show_upper_limit=True,
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)
axs[0].set_ylim([-175, 175])

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep_participant(
        results_guardian_plot,
        selected_algos_for_residual_guardian[i],
        alpha=0.5,
        show_upper_limit=True,
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)
axs[0].set_ylim([-175, 175])

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_per_participant_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep_phase(
        results_guardian_plot,
        selected_algos_for_residual_guardian[i],
        alpha=0.5,
        show_legend=(i == 0),
        show_upper_limit=True,
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
        rect=(0, 0, 0.85, 1),
    )
    if i != 0:
        axs[1].set_ylabel(None)
axs[0].set_ylim([-175, 175])

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_per_phase_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = residual_plot_pep_heart_rate(
        results_guardian_plot,
        selected_algos_for_residual_guardian[i],
        alpha=0.5,
        show_legend=(i == 0),
        show_upper_limit=True,
        annotate_fontsize="small",
        annotate_bbox=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)
axs[0].set_ylim([-175, 175])

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_residual_plots_b_point_algorithms_heart_rate_guardian.pdf"), transparent=True)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

for i, ax in enumerate(axs):
    fig, ax = regplot_error_heart_rate(
        results_guardian_plot,
        selected_algos_for_residual_guardian[i],
        error_metric="absolute_error_per_sample_ms",
        add_corr_coeff=True,
        ax=ax,
    )
    if i != 0:
        axs[1].set_ylabel(None)

fig.tight_layout()

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_b_point_error_heart_rate_guardian.pdf"), transparent=True)

#### Effect of Outlier Correction on Output Error

In [ ]:
metrics_guardian_outlier = create_outlier_correction_table(metrics_guardian)
metrics_guardian_outlier_style = metrics_guardian_outlier.style.apply(
    styling.highlight_outlier_improvement, subset=["Mean Absolute Error [ms]", "Invalid PEPs"]
)
metrics_guardian_outlier_style

In [ ]:
latex_output = convert_to_latex(
    metrics_guardian_outlier_style,
    collapse_index_columns=False,
    column_header_bold=True,
    siunitx=True,
    convert_css=True,
    column_format="p{1.5cm}p{1.5cm}"
    + ("S[table-column-width=0.75cm]" * (len(metrics_empkins_outlier_style.columns) - 3))
    + "p{1.0cm}" * 3,
    caption=r"Effect of Outlier Correction algorithms on the B-point extraction algorithms for the \textit{Guardian Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order. Resuls highlighted in \textcolor{LightGreen}{green} indicate an improvement of the metric through outlier correction, \textcolor{Pink}{red} indicate no improvement.",
    label="tab:outlier_correction_results_full_guardian",
)

latex_output = latex_output.replace(r"\sisetup{", r"\sisetup{round-mode=places,round-precision=2,")
latex_output = latex_output.replace(r"\bfseries \bfseries", r"\bfseries")
latex_output = latex_output.replace(r"\bfseries \bfseries", r"\bfseries")
latex_output = latex_output.replace(r"\bfseries \bfseries", r"\bfseries")
latex_output = latex_output.replace(r"\multicolumn{2}{r}", r"\multicolumn{2}{c}")
latex_output = latex_output.replace(r"Mean Absolute Error [ms]", r"MAE [ms]")
latex_output = latex_output.replace(r"Mean Error [ms]", r"ME [ms]")
latex_output = latex_output.replace(r"Mean Absolute Relative Error [\%]", r"MARE [\%]")
latex_output = latex_output.replace(r"{B-Point Detection}", r"{B-Point\newline Detection}")

suppl_tab_path_paper.joinpath("tab_outlier_correction_results_full_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Horizontal Table

In [ ]:
metrics_guardian_outlier_unstack = metrics_guardian_outlier[["Mean Absolute Error [ms]"]]
metrics_guardian_outlier_unstack = (
    metrics_guardian_outlier_unstack.unstack(sort=False).reorder_levels([0, 2, 1], axis=1).sort_index(axis=1)
)
metrics_guardian_outlier_unstack = metrics_guardian_outlier_unstack.reindex(outlier_algos_rename, level=1, axis=1)
metrics_guardian_outlier_unstack

##### To LaTeX

In [ ]:
result_table = create_algorithm_result_table(metrics_guardian[["Mean Absolute Error [ms]"]])
result_table = result_table.unstack("Outlier Correction").reindex(result_table.xs("None", level=-1).index, level=0)
result_table = result_table.reindex(outlier_algos_rename, level="Outlier Correction", axis=1)

latex_output = convert_to_latex(
    result_table.style.apply(styling.highlight_min_uncertainty, axis=1),
    collapse_index_columns=False,
    column_header_bold=True,
    siunitx=False,
    column_format="p{1.0cm}p{1.5cm}p{1.5cm}p{1.5cm}",
    caption=r"\ac{MAE} of the Outlier Correction algorithms on the B-point extraction algorithms on the \textit{Guardian Dataset}. The algorithms are sorted by the \acf{MAE} in ascending order. The lowest \ac{MAE} values per algorithm are highlighted in \textbf{bold}. \ac{MAE} values are provided in milliseconds as (\(M\,\pm\,SD\)).",
    label="tab:outlier_correction_results_guardian",
)

# some manual post-processing
latex_output = latex_output.replace(
    r"\multicolumn{3}{r}{\bfseries Mean Absolute Error [ms]}",
    r"\multicolumn{3}{l}{\bfseries Outlier Correction Algorithm}",
)
latex_output = latex_output.replace(r"Outlier Correction & ", r"{\bfseries B-point Algorithm} & ")
latex_output = latex_output.replace(r"B-Point Detection &  &  &  \\", r"")
latex_output = latex_output.replace(r" \pm ", r"\(\pm\)")

tab_path_paper.joinpath("tab_outlier_correction_results_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

#### Outlier Correction Residual Plots

In [ ]:
for algo in selected_algos_for_residual_outlier_correction_guardian:
    fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=True)

    results_per_algorithm_plot_guardian = results_per_sample_guardian.xs(
        algo, level="b_point_algorithm", drop_level=False
    )

    for i, outlier_algo in enumerate(outlier_algos):
        selected_algo = [algo, outlier_algo]
        residual_plot_pep(
            results_per_sample_guardian, selected_algo, ax=axs[i], show_upper_limit=True, annotate_fontsize="small"
        )
        if i != 0:
            axs[i].set_ylabel(None)
        axs[i].set_ylim([-150, 175])

    fig.tight_layout()
    for path in [img_path, suppl_img_path_paper]:
        fig.savefig(
            path.joinpath(f"img_residual_plots_b_point_outlier_correction_{algo}_guardian.pdf"), transparent=True
        )

In [ ]:
dv = "absolute_error_per_sample_ms"

for algo in selected_algos_for_residual_outlier_correction_guardian:
    results_per_algorithm_plot_guardian = results_per_sample_guardian.xs(algo, level="b_point_algorithm")
    data_plot_paired = add_unique_id_to_results_dataframe(results_per_algorithm_plot_guardian[[dv]])

    outlier_algo_combis = [(outlier_algos[0], outlier_algos[1]), (outlier_algos[0], outlier_algos[2])]

    fig, axs = plt.subplots(ncols=2, figsize=(4, 4), sharey=True)
    fig, axs = paired_plot_error_outlier_correction(
        data=data_plot_paired, outlier_algo_combis=outlier_algo_combis, dv=dv, title=algo, axs=axs
    )

    display(Markdown(f"**B-point Algorithm**: {algo}"))
    for outlier_algo in outlier_algo_combis:
        display(Markdown(f"""**Outlier Correction Algorithms**: {' vs. '.join(outlier_algo)}"""))
        display(compute_improvement_outlier_correction(data_plot_paired, outlier_algo))

    fig.tight_layout()
    for path in [img_path, suppl_img_path_paper]:
        fig.savefig(
            path.joinpath(f"img_paired_plot_b_point_outlier_correction_improvement_{algo}_guardian.pdf"),
            transparent=True,
        )

### `NaN` Reason Table

In [ ]:
nan_reason_table_guardian = create_nan_reason_table(
    results_per_sample_guardian, outlier_algos=outlier_algos, use_short_names=True
)
nan_reason_table_guardian.head()

##### To LaTeX

In [ ]:
latex_output = convert_to_latex(
    nan_reason_table_guardian,
    column_format="p{1.5cm}" * 2 + "p{1.0cm}" * len(nan_reason_table_empkins.columns),
    column_header_bold=True,
    escape_columns=True,
    caption=r"Overview of invalid PEP reasons for different B-point algorithms on the \textit{Guardian Dataset}. Abbreviations: "
    + ", ".join([rf"\textit{{{k}}}: {v}" for k, v in get_nan_reason_mapping().items()]),
    label="tab:nan_reasons_guardian",
)

# some manual post-processing
latex_output = latex_output.replace(r"\centering", r"\small\centering")
latex_output = latex_output.replace(r"{Reason}", r"{\bfseries Reason}")

suppl_tab_path_paper.joinpath("tab_b_point_nan_reason_guardian.tex").open(mode="w+").write(latex_output)
print(latex_output)

## Combined Figures

### Absolute Error - EmpkinS and Guardian

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 2), sharey=True)

boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="EmpkinS Dataset – B-Point Detection",
    fig=fig,
    ax=axs[0],
)
boxplot_algorithm_performance(
    results_guardian_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    width=0.9,
    title="Guardian Dataset – B-Point Detection",
    fig=fig,
    ax=axs[1],
)
for ax in axs:
    ax.set_xlabel(None)
axs[1].set_ylabel(None)

fig.tight_layout()

for path in [img_path, img_path_paper]:
    fig.savefig(path.joinpath("img_boxplot_b_point_algorithms_mae_combined.pdf"), transparent=True)

### Outlier Correction - Residual and Paired Plots (Guardian)

In [ ]:
selected_algo = "forouzanfar2018"
dv = "absolute_error_per_sample_ms"

fig, axs = plt.subplots(ncols=3, figsize=(12, 4), sharey=False)
selected_results_guardian = results_per_sample_guardian.xs(selected_algo, level="b_point_algorithm", drop_level=True)
selected_outlier_algos = ["none", "forouzanfar2018"]
for outlier_algo, ax in zip(selected_outlier_algos, axs, strict=False):
    residual_plot_pep(selected_results_guardian, outlier_algo, ax=ax, show_upper_limit=True)
    ax.set_ylim([-150, 175])

axs[1].set_ylabel(None)
axs[1].set_yticklabels([])

selected_results_guardian = results_per_sample_guardian.xs(selected_algo, level="b_point_algorithm", drop_level=True)
data_plot_paired = add_unique_id_to_results_dataframe(selected_results_guardian[[dv]])
paired_plot_error_outlier_correction(
    data=data_plot_paired, outlier_algo_combis=[selected_outlier_algos], dv=dv, axs=[axs[-1]]
)

axs[-1].set_title("Effect of Outlier Correction\n on Absolute Error", fontweight="bold")
axs[-1].set_xlabel("Outlier Correction Algorithm")

display(Markdown(f"""**Outlier Correction Algorithms**: {' vs. '.join(selected_outlier_algos)}"""))
display(compute_improvement_outlier_correction(selected_results_guardian[[dv]], selected_outlier_algos))

fig.tight_layout()

for path in [img_path, img_path_paper]:
    fig.savefig(path.joinpath("img_effect_outlier_correction_guardian_combined.pdf"), transparent=True)